In [1]:
import ipywidgets as widgets

import random
from poker import Card, Suit, Rank

import pandas as pd
import numpy as np


In [2]:
class Player(object):
    
    def __init__(self):
        self.hand = [] 
        self.numCards = 2
        self.calls = []
        self.calledBluff = False
        
    def makeList(self):
        allPlayers = {}
        numPl = int(input('Enter number of players: '))
        
        if numPl >= 2 and numPl < 7:
            for e in range(numPl):
                name = "Player" + str(e+1)
                allPlayers[name] = {'Hand':Player().hand , 
                                    'Calls':Player().calls,
                                    'Number of Cards' : Player().numCards, 
                                    'Called Bluff?' : Player().calledBluff}
            return allPlayers
        else:
            print("Invalid number of players. Enter a number between 2 and 6.")
            return Player().makeList()


# Save to a DataFrame then onto a CSV

In [3]:
class Save:
    
    df = pd.DataFrame()
    breakLine = pd.Series(name='Round finished. ', dtype=object)
    count = int() #count restarts whenever Save class is ran. How to keep count from restarting? 
     
    def store(round_data): #call when round is over
        if Save.df.empty:
            Save.df = pd.DataFrame(round_data)
            Save.df.drop(["Number of Cards"], inplace = True)
            Save.df = Save.df.transpose()
            
            Save.df = Save.df.append(Save.breakLine)
            Save.df=Save.df.replace(np.NaN, '--')
        else:
            df2 = pd.DataFrame(round_data)
            df2.drop(["Number of Cards"], inplace = True)
            df2 = df2.transpose()
            
            Save.df = Save.df.append(df2)
            Save.df = Save.df.append(Save.breakLine)
            Save.df = Save.df.replace(np.NaN, '--')
            
        display(Save.df)

    def toCSV(): #call when game is over
        name, csv = 'game', '.csv'
        file_name = name + str(Save.count) + csv
        Save.count += 1
        Save.df.to_csv(file_name)
    
# for each round, append onto the cvs file. 
# as rounds keep piling up, number of players should decrease
# after each game, close cvs file. a new dataframe/file should be created for another game.     

# Helper Functions

In [4]:
class Helper:
    
    def getSuits(self):
        handSuits = []
        for card in self:
            handSuits.append(card.suit)
        return handSuits

    def getRanks(self):
        handRanks = []
        for card in self:
            if card.rank == Rank('T'):
                card.rank = 10
            elif card.rank == Rank('J'):
                card.rank = 11
            elif card.rank == Rank('Q'):
                card.rank = 12
            elif card.rank == Rank('K'):
                card.rank = 13
            elif card.rank == Rank('A'):
                card.rank = 14
            handRanks.append(int(str(card.rank)))
        return handRanks
    
    def setSuits(self):
        print('\nSPADES = s, HEARTS = h, DIAMONDS = d, CLUBS = c\n')
        suitInput = input('Suit: ')
        if suitInput != 's' or suitInput != 'h' or suitInput != 'd' or suitInput != 'c':
            print('Input not valid. Enter "s", "h", "d", or "c".')
            return Helper().setSuits()
        else:
            return Suit(suitInput)
    
    def setRanks(self):
        print('\n2, 3, 4, 5, 6, 7, 8 , 9, T, J, Q, K, A\n')
        rankInput = input('Rank: ').upper()
        
        return Rank(rankInput)
    
    def setHand(self, players):
        calledHand = [None]
        
        display(hand)
        print("Or enter '0' to call bluff.\n")
        handInput = int(input('Call: '))
        
        if handInput > 9:
            print('Input not valid. Enter a number between 0 and 9.')
            return Helper().setHand(players)
        elif handInput == 0:
            print('Bluff called. Combine cards...')
        else:
            calledHand.insert(0, handInput)
            if handInput == 3 or handInput == 7:
                inputR = Helper().setRanks()
                calledHand.insert(2,inputR)
                print('Enter 2nd rank.')
                inputR2 = Helper().setRanks()
                calledHand.insert(3,inputR2)
            elif handInput == 6:
                inputS = Helper().setSuits()
                calledHand.insert(1, inputS)
            elif handInput == 9:
                inputS = Helper().setSuits()
                calledHand.insert(1, inputS)
                inputR = Helper().setRanks()
                calledHand.insert(2, inputR)
            else:
                inputR = Helper().setRanks()
                calledHand.insert(2, inputR)
        return calledHand

    def numOfSameCard(self):
        ranks = Helper.getRanks(self)
        sameRank = max(ranks.count(2),ranks.count(3),ranks.count(4),ranks.count(5),
                     ranks.count(6),ranks.count(7),ranks.count(8),ranks.count(9),
                     ranks.count(10),ranks.count(11),ranks.count(12),ranks.count(13),
                     ranks.count(14))
        return sameRank
    
    def removeDup(self):
        rd = list(dict.fromkeys(self))
        rd.sort()
        return rd
    
    def stringCall(self, call):
        
        if call[0] == 3 or call[0] == 7:
            str_hand = hand[call[0]]
            str_rank = str(Rank(call[2]))
            str_rank2 = str(Rank(call[3]))
            return str_hand + ' of  ' + str_rank + ' & ' + str_rank
        elif call[0] == 5:
            str_hand = hand[call[0]]
            str_rank = str(Rank(call[2]))
            return str_hand + ' to ' + str_rank
        elif call[0] == 6:
            str_hand = hand[call[0]]
            str_suit = str(Suit(call[1]))
            return str_hand + ' of ' + str_suit
        elif call[0] == 9:
            str_hand = hand[call[0]]
            str_suit = str(Suit(call[1]))
            str_rank = str(Rank(call[2]))
            return str_hand + ' of ' + str_suit + ' to ' + str_rank
        else:
            str_hand = hand[call[0]]
            str_rank = str(Rank(call[2]))
            return str_hand + ' of ' + str_rank
    

# Game Functions

In [9]:
class Game:
    
    global deck, suit, rank, hand
    deck = list(Card)
    suit = list(Suit)
    rank = list(Rank)
    hand = {1:'High Card',2:'Pair',3:'Two Pair',4:'Three of a Kind',5:'Straight',6:'Flush',7:'Full House',8:'Four of a Kind',9:'Straight Flush'}
    prevCall = [None]
        
    def shuffleDeck():
        random.shuffle(deck)

    def deal():
        return deck.pop(0)
    
    def dealToAllPlayers(players):
        count = 1
        for player in players:
            for e in range(players[player]['Number of Cards']):
                if players[player]['Number of Cards'] != 6:
                    players[player]['Hand'].append(Game.deal())
            count += 1
            
    def combineCards(players):
        allCards = []
        for e in players:
            allCards = allCards + players[e]['Hand']
        return allCards
    
    def handCall(players):
        currCall = Helper().setHand(players)
        if currCall == [None]:
            return Game.bluffCall(players)
        else:
            pass
        
        if Game.prevCall == [None]:
            Game.prevCall = currCall
            return currCall
         
        elif currCall[0] > Game.prevCall[0]: 
            Game.prevCall = currCall
            return currCall
        
        elif currCall[0] == Game.prevCall[0]:
            if currCall[1] == None:
                if currCall[2] > Game.prevCall[2]:
                    Game.prevCall = currCall
                    return currCall
                else:
                    print('\nCall needs to be higher than the previous call. The previous call was:')
                    display(Helper().stringCall(Game.prevCall))
                    return Game.handCall(players)
            else: 
                if currCall[1] > Game.prevCall[1]:
                    Game.prevCall = currCall 
                    return currCall
                else: 
                    print('\nCall needs to be higher than the previous call. The previous call was:')
                    display(Helper().stringCall(Game.prevCall))
                    return Game.handCall(players)
        
        elif currCall[0] < Game.prevCall[0]: 
            print('\nCall needs to be higher than the previous call. The previous call was:')
            display(Helper().stringCall(Game.prevCall))
            return Game.handCall(players)

    def bluffCall(players):
        pool = Game.combineCards(players)
        
        if Game.prevCall[0] == 1:
            check = CheckHands.checkHigh(pool, Game.prevCall[2])
        elif Game.prevCall[0] == 2:  
            check = CheckHands.checkPair(pool, Game.prevCall[2])
        elif Game.prevCall[0] == 3:
            check = CheckHands.check2Pair(pool, Game.prevCall[2],Game.prevCall[3])
        elif Game.prevCall[0] == 4:
            check = CheckHands.check3Kind(pool, Game.prevCall[2])
        elif Game.prevCall[0] == 5:
            check = CheckHands.checkStraight(pool, Game.prevCall[2])
        elif Game.prevCall[0] == 6:
            check = CheckHands.checkFlush(pool, Game.prevCall[1])
        elif Game.prevCall[0] == 7:
            check = CheckHands.checkFullHouse(pool, Game.prevCall[2], Game.prevCall[3])
        elif Game.prevCall[0] == 8:
            check = CheckHands.check4Kind(pool, Game.prevCall[2])
        elif Game.prevCall[0] == 9:
            check = CheckHands.checkStraightFlush(pool, Game.prevCall[1], Game.prevCall[2])
        
        if check == False:
            print('\nPool does not contain the call. Hand caller gets an additional card.')
            return False
        else:
            print('\nPool contains the call. Bluff caller gets an additional card.')
            return True
        
    def startRound(players,round_over):
        prevPlayer = ''
        while not round_over:
            for player in players:
                #get hand calls
                display(player)
                print('Hand: ' + str(players[player]['Hand']))
                call = Game.handCall(players)
                
                #check if player called hand or called bluff
                if type(call) == list:
                    players[player]['Calls'].append(Helper().stringCall(call))
                    prevPlayer = player
                else:
                    players[player]['Called Bluff?'] = True
                    if call == True: 
                        #add card to bluff caller
                        players[player]['Number of Cards'] += 1
                        if players[player]['Number of Cards'] == 6:
                            del players[player]
                    else:
                        #add card to hand caller instead
                        players[prevPlayer]['Number of Cards'] += 1
                        if players[prevPlayer]['Number of Cards'] == 6:
                            del players[prevPlayer]          
                    #round over, append round data into Game data file.             
                    print('Round finished.')
                    Save.store(players)
                    round_over = True
                    break
                    
    def roundReset(self,players):  
        for player in players:
            players[player]['Hand'] = []
            players[player]['Calls'] = []
            players[player]['Called Bluff?'] = False
        deck = list(Card)
    
    def run():
        #set number of players and create list 
        players = Player().makeList()
        
        #start game
        game_over = False
        round_over = False
        round_count = 1
        while not game_over:
            if len(players) != 1:
                print('\nRound ' + str(round_count) + ':')    
                round_count += 1
                
                Game.prevCall = [None]
                Game().roundReset(players)
                
                #shuffle deck and distribute
                Game.shuffleDeck()
                Game.dealToAllPlayers(players)
                
                Game.startRound(players,round_over)                
            else:
                print('Game over.')
                print(list(players.keys())[0] + 'won!')
                Save.toCSV()
                game_over = True


# Check Poker Hands

In [6]:
class CheckHands:
    
    def removeTwo(self):   #Used for the straight function Mayber
        for card in self:
            if card == 2:
                pos = self.index(card)
                self.pop(pos)
        return self
    
    def countTwo(self):
        self = Helper.getRanks(self)
        count = 0
        for card in self:
            if card == 2:
                count += 1
        return count
    
    def checkHigh(self, inpRank):
        self = Helper.getRanks(self)
        
        if self.count(inpRank) > 0:
            return True
        else: 
            return False
    
    def checkFlush(self, inpSuit):
        hand = self
        self = Helper.getSuits(self)
        check = Suit(inpSuit)
        
        s = suit[3]
        h = suit[2]
        d = suit[1]
        c = suit[0]
        num = max(self.count(s),self.count(h),self.count(d),self.count(c))
        if num == self.count(s):
            maxSuit = s
        elif num == self.count(h):
            maxSuit = h
        elif num == self.count(d):
            maxSuit = d
        elif num == self.count(c):
            maxSuit = c
        if (num < 5) or check != maxSuit:
            numTwo = CheckHands.countTwo(hand)
            for card in hand:
                if '2' + inpSuit == str(card):
                    numTwo = numTwo - 1
            if numTwo + num < 5:
                return False, maxSuit
            else:
                return True, maxSuit
        else:
            return True, maxSuit
     
    def checkStraight(self, high):
        numTwo = CheckHands.countTwo(self)
        self = Helper.getRanks(self) #remove duplicates and sort
        self = CheckHands.removeTwo(self)
        if low in self:
            pass
        elif numTwo > 0:
            numTwo -= 1
            self.insert(0, low)
        if low in self:
            straight = list(range(int(low),high+1))
            count = 0
            inARow = 0
            for num in straight:
                if num in self:
                    inARow += 1
                elif numTwo > 0:
                    numTwo -= 1
                    inARow += 1
                count += 1
            if inARow < 5:
                return False
            else:
                return True
        else:
            return False
        
    def checkStraightFlush(self, suit, high):
        isFlush, maxSuit = CheckHands.checkFlush(self, suit)
        if isFlush:
            flushRanks = []
            i = 0
            for e in Helper.getSuits(self):
                if e == maxSuit or Helper.getRanks(self)[i] == 2:
                    flushRanks.append(self[i])
                i += 1
            if CheckHands.checkStraight(flushRanks, low, high):
                return True
            else:
                return False
        else:
            return False
        
    def checkFourKind(self, inpQuad):
        ranks = Helper.getRanks(self)
        if ranks.count(inpQuad) == 4:
            return True
        else:
            return False
        
    def check3Kind(self, inpTriple):
        ranks = Helper.getRanks(self)
        if ranks.count(inpTriple) >= 3:
            return True
        else:
            return False
    
    def checkPair(self, inpDouble):
        ranks = Helper.getRanks(self)
        if ranks.count(inpDouble) >=2:
            return True
        else:
            return False
        
    def checkFullHouse(self, inpTriple, inpDouble):
        if inpTriple == inpDouble:
            return False
        if CheckHands.check3Kind(self, inpTriple) and CheckHands.checkPair(self, inpDouble):
            return True
        else:
            return False
    
    def check2Pair(self, inpDouble, inpDoubleTwo):
        if inpDouble == inpDoubleTwo:
            return False
        if CheckHands.checkPair(self, inpDouble) and CheckHands.checkPair(self, inpDoubleTwo):
            return True
        else:
            return False

# Get Probabilities

In [7]:
class Heuristic:
    
    def getRandomHand(numCards, myHand):
        sampleDeck = list(Card)
        random.shuffle(sampleDeck)
        cards = []
        #remove cards in deck that are in your hand
        if myHand == []:
            pass
        else:
            for card in myHand:
                loc = sampleDeck.index(card)
                cards.append(sampleDeck.pop(loc))
        
        for num in range(numCards - len(myHand)):
            currCard = sampleDeck.pop(0)
            cards.append(currCard)
        return cards
    
    def probGeneral(numCards, trials, myHand):
        numStraightFlush = 0
        numFourKind = 0
        numFullHouse = 0
        numFlush = 0
        numStraight = 0
        numThreeKind = 0
        numTwoPair = 0
        numPair = 0
        
        def straightFlush(self):     #Test number of Straight Flushes
            for num in range(6,14):   
                for suit in list(Suit):
                    if (CheckHands.checkStraightFlush(self, str(suit), num - 4, num)):
                        return 1
            return 0
        
        def fourKind(self):#Test number of 4 of a Kind
            for num in range(2,14):   
                if (CheckHands.checkFourKind(self, num)):
                    return 1
            return 0
        
        def fullHouse(self):#Test number of Full Houses
            for num in range(2, 14):  
                if not CheckHands.check3Kind(self, num):
                    continue
                for e in range(2, 14):
                    if CheckHands.checkFullHouse(self, num, e):
                        return 1
            return 0
        
        def Flush(self): # Test Number of Flushes
            for suit in list(Suit):
                isFlush, maxSuit = CheckHands.checkFlush(self, str(suit))
                if isFlush:
                    return 1
            return 0
        
        def Straight(self): # Test Number of Straights
            for num in range(6,14):
                if (CheckHands.checkStraight(self, num - 4, num)):
                    return 1
            return 0
        
        def ThreeKind(self):
            for num in range(2,14):
                if CheckHands.check3Kind(self, num):
                    return 1
            return 0
        
        def TwoPair(self):
            for num in range(2, 14):  
                if not CheckHands.checkPair(self, num):
                    continue
                for e in range(2, 14):
                    if CheckHands.check2Pair(self, num, e):
                        return 1
            return 0
        
        def Pair(self):
            for num in range(2,14):
                if CheckHands.checkPair(self, num):
                    return 1
            return 0
        
        for trial in range(trials):
            handTrial = Heuristic.getRandomHand(numCards, myHand)
            
            numStraightFlush += straightFlush(handTrial)
            numFourKind += fourKind(handTrial)
            numFullHouse += fullHouse(handTrial)
            numFlush += Flush(handTrial)
            numStraight += Straight(handTrial)
            numThreeKind += ThreeKind(handTrial)
            numTwoPair += TwoPair(handTrial)
            numPair += Pair(handTrial)
            
        print ("Straight Flush Probability:", numStraightFlush/trials)
        print ("4 of a Kind Probability:", numFourKind/trials)
        print ("Full House Probability:", numFullHouse/trials)
        print ("Flush Probability:", numFlush/trials)
        print ("Straight Probability:", numStraight/trials)
        print ("3 of a Kind Probability:", numThreeKind/trials)
        print ("2 pair Probability:", numTwoPair/trials)
        print ("Pair Probability:", numPair/trials)

In [ ]:
Game.run()